In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import timeit
import pickle
import sys
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, \
                            precision_recall_curve, roc_curve, accuracy_score
from sklearn.exceptions import NotFittedError

In [2]:
cd /content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner

/content/drive/.shortcut-targets-by-id/1PND44K5yRW8mjP-NB4mfwEivBcL8QvfE/2021_VIIT08_P300/Dataset/bci-ner


In [3]:
!pip install -U imbalanced-learn
!pip install pyriemann
!pip install sklearn

Requirement already up-to-date: imbalanced-learn in /usr/local/lib/python3.7/dist-packages (0.8.0)


In [5]:
import pandas as pd
import random
from pyriemann.estimation import XdawnCovariances, Covariances
from pyriemann.tangentspace import TangentSpace
from os.path import isfile
import numpy as np
import h5py
from imblearn.over_sampling import RandomOverSampler

In [6]:
X_train=np.load('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/train_data.npy')
X_test=np.load('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/test_data.npy')
Y_test = np.reshape(pd.read_csv('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/true_labels.csv', header=None).values, 3400)

Y_train = np.load ('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/Y_train.npy')

In [7]:
np.count_nonzero(Y_train == 0),np.count_nonzero(Y_train == 1)

(1590, 3850)

In [8]:
pip install numpy matplotlib scipy numba scikit-learn mne PyWavelets pandas

In [9]:
pip install mne-features

In [10]:
from mne_features.feature_extraction import extract_features
from mne_features.univariate import get_univariate_funcs

In [11]:
X_train.shape
X_train=np.reshape(X_train,(16*340, 56,140))

In [12]:
X_train.shape

(5440, 56, 140)

In [14]:
params1=['mean','variance','std']
new_X1=extract_features(X_train,128, params1 )
new_X1.shape

(5440, 168)

In [ ]:
params2=['higuchi_fd','hurst_exp']
new_X2=extract_features(X_train,128, params2 )
new_X2.shape

(5440, 112)

In [ ]:

params3=['spect_entropy','ptp_amp']
new_X3=extract_features(X_train,128, params3 )
new_X3.shape

(5440, 112)

In [ ]:
params4 =['wavelet_coef_energy']
new_x4 =extract_features(X_train,128, params4 )
new_x4.shape

(5440, 224)

In [ ]:
XC = XdawnCovariances(nfilter=4,estimator='oas',xdawn_estimator="oas") # our transformer
TS = TangentSpace(metric='riemann')

X_train_newX = XC.fit_transform(X_train,Y_train)
# X_train = fit_transform(train_data)
print("Xdawn Covariances", X_train_newX.shape)

Xdawn Covariances (5440, 16, 16)


In [ ]:
X_train_newX = TS.fit_transform(X_train_newX)
print("Tangent Spaces", X_train_newX.shape)

Tangent Spaces (5440, 136)


In [ ]:
boss= np.concatenate((new_X1,new_X3,new_X2,new_x4,X_train_newX),axis=1)

In [ ]:
boss.shape

(5440, 752)

Same with X_test

In [ ]:
X_test.shape
X_test=X_test.reshape(10*340, 56,140)

In [ ]:
params1=['mean','variance','std']
new_X1=extract_features(X_test,128, params1 )
new_X1.shape

In [ ]:
params2=['higuchi_fd','hurst_exp']
new_X2=extract_features(X_test,128, params2 )
new_X2.shape

In [ ]:
params3=['spect_entropy','ptp_amp']
new_X3=extract_features(X_test,128, params3 )
new_X3.shape

In [ ]:
params4 =['wavelet_coef_energy']
new_x4 =extract_features(X_test,128, params4 )
new_x4.shape

In [ ]:
X_test = XC.transform(X_test)
# X_test = Covariances('oas').fit_transform(X_test)
X_test = TS.transform(X_test)


In [ ]:
X_test_new= np.concatenate((new_X1,new_X3,new_X2,new_x4, X_test),axis=1)

Applying LightGBM

In [ ]:
pip install lightgbm

In [ ]:
from lightgbm import LGBMClassifier
import optuna.integration.lightgbm as lgb

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
lgbm = LGBMClassifier()

In [ ]:
lgbm.fit(boss,Y_train)

In [ ]:
y_preds=lgbm.predict(X_test_new)
cr = classification_report(Y_test,y_preds)
print(cr)

In [ ]:
confusion_matrix(Y_test,y_preds)

Hyper-Parameter Tuning using Optuna

In [ ]:
pip install scikit-learn

In [ ]:
pip install optuna

In [ ]:
import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler
# from a import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [ ]:
def objective(trial) -> float:
    
    # joblib.dump(study, 'study.pkl')
    
    # train_X,test_X,train_y,test_y = train_test_split(X, Y, test_size = 0.30,random_state = 101)
    # dtrain = lgb.Dataset(X_train, label=y_train)
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
           }
 
    
    lgbm.fit(boss,Y_train)
    preds = lgbm.predict(X_test_new)
    # pred_labels = np.rint(preds)
    accuracy = accuracy_score(Y_test, preds)
    return accuracy    lgbm = LGBMClassifier(**param)

    

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

In [ ]:
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
best_params, tuning_history = dict(), list()
booster = lgbm.train(best_param, dtrain, valid_sets=dval,
                    verbose_eval=0,
                    best_params=best_params,
                    tuning_history=tuning_history)
 
print("Best Params:", best_params)
print("Tuning history:", tuning_history)

In [ ]:
lgbm = LGBMClassifier(**study.best_trial.params)
lgbm.fit(boss,Y_train)

In [ ]:
y_preds=lgbm.predict(X_test_new)
from sklearn.metrics import classification_report,confusion_matrix
cls = classification_report(Y_test,y_preds)
print(cls)
print("------------COnfusion Matrix---------------")
confusion_matrix(Y_test,y_preds)

In [ ]:
optuna.visualization.plot_param_importances(study)

Optuna with hyperparameter importance threshold

In [ ]:
def objective(trial) -> float:
    
    # joblib.dump(study, 'study.pkl')
    
    # train_X,test_X,train_y,test_y = train_test_split(X, Y, test_size = 0.30,random_state = 101)
    # dtrain = lgb.Dataset(X_train, label=y_train)
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        # 'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        # 'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
 
    
    lgbm = LGBMClassifier(**param)
    lgbm.fit(boss,Y_train)
    preds = lgbm.predict(X_test_new)
    # pred_labels = np.rint(preds)
    accuracy = accuracy_score(Y_test, preds)
    return accuracy

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

In [ ]:
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
lgbm = LGBMClassifier(**study.best_trial.params)
lgbm.fit(boss,Y_train)

In [ ]:
y_preds=lgbm.predict(X_test_new)
from sklearn.metrics import classification_report,confusion_matrix
cls = classification_report(Y_test,y_preds)
print(cls)
print("------------COnfusion Matrix---------------")
confusion_matrix(Y_test,y_preds)

In [ ]:
optuna.visualization.plot_param_importances(study)